# UV_LED_GRID_FOR_PHOTOLITHOGRAPGHY

### Libraries 

In [49]:
from multiprocessing import Pool
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

### Functions 

In [50]:
def arrangement_1(nleds,d):
    l=math.sqrt(nleds);
    ledcord=np.zeros((2,nleds));
    temp=0;
    for k in range(nleds):
        ledcord[0,k]=-(l-1)*d/2 +((k)%l)*d;
        if (k)%l ==0 and k != 0:
            temp=temp+1;
        ledcord[1,k]=-(l-1)*d/2 + temp*d;
    return ledcord

def arrangement_2(nleds,d):
    l=math.sqrt(nleds);
    ledcord=np.zeros((2,nleds));
    temp=0;
    for k in range(nleds):
        if (k)%2 ==0:
            ledcord[0,k]=-(l-1)*d/2 +((k)%l)*d;
        elif (k)%2 ==1:
            ledcord[0,k]=-(l-2)*d/2 +((k)%l)*d;
        
        if (k)%l ==0 and k != 0:
            temp=temp+1;
        
        ledcord[1,k]=-(l-1)*d/2 + temp*d;
    return ledcord

def E_r(ledcord,nleds,Ir_0,z,d,r,th):    
#     x=r*np.cos(th);
    x=-r;
    y=0
#     y=r*np.sin(th);
    E=0;
    for i in range(nleds):
        den=np.square(np.square(x-ledcord[0,i]) +np.square(y-ledcord[1,i]) +z**2);
        E= E + (z**2 * Ir_0)*(np.reciprocal(den))
    return E

# Arrangement 1 

In [11]:
Ir_0 = 0.1
final_list = []

def ratio(d):
    final_list = []
    Ir_0=0.1
    for nleds in [9,25,49,81,121]:
        lled=np.sqrt(nleds)   
        ledcord = arrangement_1(nleds,d)
        nz = round((0.5 - d)/0.001) + 1
        for z in np.linspace(d,0.5,nz):
            min_E=1000000;
            max_E=0;
            ni = round((0.04)/0.001) + 1
            for i in np.linspace(0,0.04,ni):
                E_i = E_r(ledcord,nleds,Ir_0,z,d,i,th)
                min_E = min(min_E,E_i)
                max_E = max(max_E,E_i)
            percent_uniformity=(min_E/max_E)*100

            t = 4000/(max_E + min_E)
            energy_max = max_E * t
            energy_min = min_E * t

            final_list.append([nleds,d,z,min_E,max_E,t,energy_min/10,energy_max/10,percent_uniformity])
    return final_list

with Pool(7) as p:
    l = list(tqdm(p.imap(ratio,np.linspace(0.01,0.2,191)), total=191))

100%|██████████| 191/191 [45:43<00:00, 14.36s/it] 


In [12]:
t=[]
for i in l:
    t.extend(i)
final_list=t

In [56]:
df_1=pd.read_pickle("./df_1.pkl")
#df_1 = pd.DataFrame(final_list,columns=["nleds","d(m)","z(m)","min_E(W/m^2)","max_E(W/m^2)","exposure_time(s)","energy_min(mJ/cm^2)","energy_max(mJ^cm^2)","percent_uniformity"])
# df_1.to_pickle("./df_1.pkl")
df_1

,nleds,d(m),z(m),min_E(W/m^2),max_E(W/m^2),exposure_time(s),energy_min(mJ/cm^2),energy_max(mJ^cm^2),percent_uniformity
0,9,0.01,0.010,40.384747,2444.444444,1.609769,6.501010,393.498990,1.652103
1,9,0.01,0.011,47.278162,2287.132920,1.713494,8.101086,391.898914,2.067137
2,9,0.01,0.012,54.306758,2148.675899,1.815720,9.860587,390.139413,2.527452
3,9,0.01,0.013,61.379583,2022.391955,1.919596,11.782402,388.217598,3.034999
4,9,0.01,0.014,68.413279,1904.966799,2.026979,13.867228,386.132772,3.591311
...,...,...,...,...,...,...,...,...,...
378175,121,0.20,0.496,6.744225,6.746008,296.510814,199.973571,200.026429,99.973575
378176,121,0.20,0.497,6.740394,6.742181,296.679239,199.973501,200.026499,99.973504
378177,121,0.20,0.498,6.736560,6.738350,296.847999,199.973430,200.026570,99.973434
378178,121,0.20,0.499,6.732722,6.734516,297.017092,199.973360,200.026640,99.973363


## No restrictions on LED grid size

### Maximum_percent_uniformity 

In [6]:
idx = df_1.groupby(['nleds'])['percent_uniformity'].transform(max) == df_1['percent_uniformity']
df_1[idx].sort_values("nleds")

,nleds,d(m),z(m),min_E(W/m^2),max_E(W/m^2),exposure_time(s),energy_min(mJ/cm^2),energy_max(mJ^cm^2),percent_uniformity
376975,9,0.2,0.500,2.492124,2.507334,800.086610,199.391541,200.608459,99.393387
377116,25,0.2,0.340,5.751994,5.764446,347.329552,199.783750,200.216250,99.783984
377437,49,0.2,0.360,6.480728,6.486233,308.476272,199.915095,200.084905,99.915131
377758,81,0.2,0.380,6.866439,6.869194,291.213385,199.959881,200.040119,99.959889
378078,121,0.2,0.399,7.097583,7.099109,281.755781,199.978503,200.021497,99.978505


### Minimum_exposure_time

In [7]:
ef_1=df_1[df_1['percent_uniformity']>99]
idx = ef_1.groupby(['nleds'])['exposure_time(s)'].transform(min) == ef_1['exposure_time(s)']
ef_1[idx].sort_values("nleds")

,nleds,d(m),z(m),min_E(W/m^2),max_E(W/m^2),exposure_time(s),energy_min(mJ/cm^2),energy_max(mJ^cm^2),percent_uniformity
339128,9,0.176,0.249,6.034938,6.095878,329.738733,198.995277,201.004723,99.000299
185160,25,0.092,0.138,28.695536,28.985091,69.347373,198.996007,201.003993,99.001022
90694,49,0.048,0.061,122.840000,124.072012,16.200103,199.002065,200.997935,99.007019
56612,81,0.033,0.041,270.900817,273.583297,7.346404,199.014671,200.985329,99.019502
38214,121,0.025,0.035,474.073674,478.809737,4.197785,199.005951,200.994049,99.010867


## LED Grid Size Less Than 0.3m

In [8]:
df_1_3 = df_1[(np.sqrt(df_1["nleds"])-1) * df_1["d(m)"] <= 0.3]

### Maximum_percent_uniformity 

In [9]:
idx = df_1_3.groupby(['nleds'])['percent_uniformity'].transform(max) == df_1_3['percent_uniformity']
df_1_3[idx].sort_values("nleds")

,nleds,d(m),z(m),min_E(W/m^2),max_E(W/m^2),exposure_time(s),energy_min(mJ/cm^2),energy_max(mJ^cm^2),percent_uniformity
293641,9,0.149,0.500,2.879550,2.903416,691.686591,199.174597,200.825403,99.177990
150026,25,0.075,0.500,8.399254,8.476842,237.021645,199.080491,200.919509,99.084699
93395,49,0.049,0.500,16.835624,16.998545,118.223682,199.036947,200.963053,99.041562
65933,81,0.037,0.048,214.731593,216.100719,9.284355,199.364427,200.635573,99.366441
50049,121,0.030,0.045,327.476602,329.699808,6.086646,199.323407,200.676593,99.325688


### Minimum_exposure_time

In [10]:
ef_1_3=df_1_3[df_1_3['percent_uniformity']>99]
idx = ef_1_3.groupby(['nleds'])['exposure_time(s)'].transform(min) == ef_1_3['exposure_time(s)']
ef_1_3[idx].sort_values("nleds")

,nleds,d(m),z(m),min_E(W/m^2),max_E(W/m^2),exposure_time(s),energy_min(mJ/cm^2),energy_max(mJ^cm^2),percent_uniformity
293556,9,0.149,0.415,3.832370,3.871068,519.248622,198.995302,201.004698,99.000324
149995,25,0.075,0.469,9.344227,9.438445,212.962242,198.996753,201.003247,99.001761
90694,49,0.048,0.061,122.840000,124.072012,16.200103,199.002065,200.997935,99.007019
56612,81,0.033,0.041,270.900817,273.583297,7.346404,199.014671,200.985329,99.019502
38214,121,0.025,0.035,474.073674,478.809737,4.197785,199.005951,200.994049,99.010867


### Constant z; Maximum_percent_uniformity 

In [29]:
z_cons = 0.35
df_1_3_z= df_1_3[(df_1_3["z(m)"]==z_cons) | (np.isclose(df_1_3["z(m)"],z_cons ))]
idx_z = df_1_3_z.groupby(['nleds'])['percent_uniformity'].transform(max) == df_1_3_z['percent_uniformity']
df_1_3_z[idx_z].sort_values("nleds")

,nleds,d(m),z(m),min_E(W/m^2),max_E(W/m^2),exposure_time(s),energy_min(mJ/cm^2),energy_max(mJ^cm^2),percent_uniformity
293491,9,0.149,0.35,4.859665,4.915556,409.197916,198.856483,201.143517,98.862984
149876,25,0.075,0.35,14.728768,14.936855,134.836200,198.597117,201.402883,98.606888
93245,49,0.049,0.35,30.040578,30.502605,66.068544,198.473728,201.526272,98.485288
66235,81,0.037,0.35,50.249345,51.041904,39.490084,198.435089,201.564911,98.447239
50354,121,0.030,0.35,75.306777,76.503038,26.348757,198.424001,201.575999,98.436323


### Constant z ;Minimum_exposure_time

In [31]:
ef_1_3_z=df_1_3_z[df_1_3_z['percent_uniformity']>98]
idx = ef_1_3_z.groupby(['nleds'])['exposure_time(s)'].transform(min) == ef_1_3_z['exposure_time(s)']
ef_1_3_z[idx].sort_values("nleds")

,nleds,d(m),z(m),min_E(W/m^2),max_E(W/m^2),exposure_time(s),energy_min(mJ/cm^2),energy_max(mJ^cm^2),percent_uniformity
157936,9,0.079,0.35,6.333506,6.462052,312.608468,197.990775,202.009225,98.010759
85989,25,0.046,0.35,17.576219,17.933495,112.645232,197.987728,202.012272,98.007772
56453,49,0.033,0.35,34.341445,35.035604,57.655955,197.998880,202.001120,98.018704
40429,81,0.026,0.35,56.542376,57.676366,35.020522,198.014354,201.985646,98.033874
28979,121,0.021,0.35,84.691160,86.399356,23.379437,198.003168,201.996832,98.022908


## Arrangement 2

In [57]:
Ir_0 = 0.1
final_list = []

def ratio(d):
    final_list = []
    Ir_0=0.1
    for nleds in [9,25,49,81,121]:
        lled=np.sqrt(nleds)   
        ledcord = arrangement_2(nleds,d)
        nz = round((0.5 - d)/0.001) + 1
        for z in np.linspace(d,0.5,nz):
            min_E=1000000;
            max_E=0;
            ni = round((0.04)/0.001) + 1
            for i in np.linspace(0,0.04,ni):
                E_i = E_r(ledcord,nleds,Ir_0,z,d,i,np.pi)
                min_E = min(min_E,E_i)
                max_E = max(max_E,E_i)
            percent_uniformity=(min_E/max_E)*100

            t = 4000/(max_E + min_E)
            energy_max = max_E * t
            energy_min = min_E * t

            final_list.append([nleds,d,z,min_E,max_E,t,energy_min/10,energy_max/10,percent_uniformity])
    return final_list

with Pool(7) as p:
    l = list(tqdm(p.imap(ratio,np.linspace(0.01,0.2,191)), total=191))

100%|██████████| 191/191 [47:16<00:00, 14.85s/it] 


In [58]:
t=[]
for i in l:
    t.extend(i)

final_list=t

In [59]:
#df_2=pd.read_pickle("./df_2.pkl")
df_2 = pd.DataFrame(final_list,columns=["nleds","d(m)","z(m)","min_E(W/m^2)","max_E(W/m^2)","exposure_time(s)","energy_min(mJ/cm^2)","energy_max(mJ^cm^2)","percent_uniformity"])
# df_2.to_pickle("./df_2.pkl")
df_2

,nleds,d(m),z(m),min_E(W/m^2),max_E(W/m^2),exposure_time(s),energy_min(mJ/cm^2),energy_max(mJ^cm^2),percent_uniformity
0,9,0.01,0.010,33.492489,2610.867120,1.512654,5.066253,394.933747,1.282811
1,9,0.01,0.011,39.310152,2381.402647,1.652406,6.495633,393.504367,1.650714
2,9,0.01,0.012,45.276702,2193.584233,1.786623,8.089239,391.910761,2.064051
3,9,0.01,0.013,51.318691,2037.132491,1.915295,9.829043,390.170957,2.519163
4,9,0.01,0.014,57.367815,1902.470236,2.040985,11.708685,388.291315,3.015438
...,...,...,...,...,...,...,...,...,...
378175,121,0.20,0.496,6.735580,6.741497,296.800263,199.912184,200.087816,99.912222
378176,121,0.20,0.497,6.731694,6.737634,296.971015,199.911786,200.088214,99.911825
378177,121,0.20,0.498,6.727805,6.733769,297.142080,199.911391,200.088609,99.911431
378178,121,0.20,0.499,6.723914,6.729901,297.313457,199.911000,200.089000,99.911039


## No restrictions on LED grid size

### Maximum_percent_uniformity

In [60]:
idx = df_2.groupby(['nleds'])['percent_uniformity'].transform(max) == df_2['percent_uniformity']
df_2[idx].sort_values("nleds")

,nleds,d(m),z(m),min_E(W/m^2),max_E(W/m^2),exposure_time(s),energy_min(mJ/cm^2),energy_max(mJ^cm^2),percent_uniformity
376704,9,0.2,0.229,5.705264,5.732020,349.733393,199.532136,200.467864,99.533228
377058,25,0.2,0.282,6.358899,6.368024,314.294352,199.856601,200.143399,99.856703
377401,49,0.2,0.324,6.736179,6.740007,296.819882,199.943180,200.056820,99.943196
377736,81,0.2,0.358,6.980369,6.982287,286.478439,199.972522,200.027478,99.972526
378064,121,0.2,0.385,7.153797,7.154851,279.551227,199.985276,200.014724,99.985277


### Minimum_exposure_time

In [61]:
ef_2=df_2[df_2['percent_uniformity']>99]
idx = ef_2.groupby(['nleds'])['exposure_time(s)'].transform(min) == ef_2['exposure_time(s)']
ef_2[idx].sort_values("nleds")

,nleds,d(m),z(m),min_E(W/m^2),max_E(W/m^2),exposure_time(s),energy_min(mJ/cm^2),energy_max(mJ^cm^2),percent_uniformity
296815,9,0.151,0.161,10.657916,10.765151,186.714631,198.998888,201.001112,99.003875
215268,25,0.107,0.126,24.285868,24.529093,81.942091,199.003485,200.996515,99.008425
158552,49,0.079,0.122,43.485853,43.923083,45.761911,198.999576,201.000424,99.004556
109138,81,0.056,0.104,87.332553,88.212657,22.786153,198.997291,201.002709,99.002293
73362,121,0.040,0.083,174.374510,176.128331,11.412176,198.999254,201.000746,99.004237


## LED Grid Size Less Than 0.3m

In [62]:
df_2_3 = df_2[(np.sqrt(df_2["nleds"])-1) * df_2["d(m)"] <= 0.3]

### Maximum_percent_uniformity 

In [63]:
idx = df_2_3.groupby(['nleds'])['percent_uniformity'].transform(max) == df_2_3['percent_uniformity']
df_2_3[idx].sort_values("nleds")

,nleds,d(m),z(m),min_E(W/m^2),max_E(W/m^2),exposure_time(s),energy_min(mJ/cm^2),energy_max(mJ^cm^2),percent_uniformity
293299,9,0.149,0.158,11.000441,11.114989,180.869192,198.964084,201.035916,98.969422
981,25,0.010,0.500,9.826686,9.967152,202.083092,198.580713,201.419287,98.590714
1472,49,0.010,0.500,19.200183,19.473497,103.429514,198.586561,201.413439,98.596480
1963,81,0.010,0.500,31.608512,32.055084,62.830256,198.597088,201.402912,98.606860
50504,121,0.030,0.500,41.741794,42.306594,47.591633,198.656014,201.343986,98.664985


### Minimum_exposure_time

In [64]:
ef_2_3=df_2_3[df_2_3['percent_uniformity']>99]
idx = ef_2_3.groupby(['nleds'])['exposure_time(s)'].transform(min) == ef_2_3['exposure_time(s)']
ef_2_3[idx].sort_values("nleds")

,nleds,d(m),z(m),min_E(W/m^2),max_E(W/m^2),exposure_time(s),energy_min(mJ/cm^2),energy_max(mJ^cm^2),percent_uniformity


### Constant z; Maximum_percent_uniformity 

In [65]:
z_cons = 0.35
df_2_3_z= df_2_3[(df_2_3["z(m)"]==z_cons) | (np.isclose(df_2_3["z(m)"],z_cons ))]
idx = df_2_3_z.groupby(['nleds'])['percent_uniformity'].transform(max) == df_2_3_z['percent_uniformity']
df_2_3_z[idx].sort_values("nleds")

,nleds,d(m),z(m),min_E(W/m^2),max_E(W/m^2),exposure_time(s),energy_min(mJ/cm^2),energy_max(mJ^cm^2),percent_uniformity
340,9,0.010,0.35,7.122375,7.329652,276.777778,197.131516,202.868484,97.172076
149876,25,0.075,0.35,14.448332,14.845016,136.549769,197.291644,202.708356,97.327830
93245,49,0.049,0.35,29.678505,30.415536,66.562341,197.547074,202.452926,97.576794
66235,81,0.037,0.35,49.806164,50.956639,39.697189,197.716470,202.283530,97.742248
50354,121,0.030,0.35,74.782654,76.418365,26.454848,197.836376,202.163624,97.859532


## Constant z ;Minimum_exposure_time

In [66]:
ef_2_3_z=df_2_3_z[df_2_3_z['percent_uniformity']>98]
idx = ef_2_3_z.groupby(['nleds'])['exposure_time(s)'].transform(min) == ef_2_3_z['exposure_time(s)']
ef_2_3_z[idx].sort_values("nleds")

,nleds,d(m),z(m),min_E(W/m^2),max_E(W/m^2),exposure_time(s),energy_min(mJ/cm^2),energy_max(mJ^cm^2),percent_uniformity


In [43]:
math.pi


3.141592653589793

In [44]:
np.pi


3.141592653589793